In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import json
import logging
import multiprocessing
from functools import partial

# Data processing
import pandas as pd
import numpy as np
from sklearn.model_selection import ParameterGrid

# Networks
import networkx as nx

# Modules
sys.path.insert(0, '../scripts')
from evaluate import Evaluate
from gsd import GSD

### Directory setup

In [3]:
home_dir = "../evaluation_clean/"
data_param_file = os.path.join(home_dir, "data_specs.json")
model_param_file = os.path.join(home_dir, "model_specs.json")

### Instantiate Evaluation object 

In [4]:
e = Evaluate(home_dir, data_param_file, model_param_file)

Number of datasets: 6


### Generate synthetic data, or load if file exists already

In [7]:
e.initialize_data()

04:16:15 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_1_Z_20_5_1_noise_0.npy.
04:16:16 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_2_Z_20_5_2_noise_0.npy.
04:16:16 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_1_Z_20_5_1_noise_0.2.npy.
04:16:16 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_2_Z_20_5_2_noise_0.2.npy.
04:16:16 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_1_Z_20_5_1_noise_0.4.npy.
04:16:16 - Evaluate: INFO - Wrote data matrix: ../evaluation_clean/data/data_matrices/D_5_100_2_Z_20_5_2_noise_0.4.npy.


### Run models on each dataset and save results

In [8]:
for model_params in e.model_paramlist: 
    e.run_grid(model_params)

04:16:24 - Evaluate: INFO - Running ICA1 with parameters: {'cutoff': 0.01, 'n_components': 10, 'method': 'ICA1'}
/nfs/latdata/iamjli/projects/GSD/venv/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
04:17:21 - Models: INFO - Run complete, results written to: ../evaluation_clean/results/ICA1_10_0.01/D_5_100_2_Z_20_5_2_noise_0.npy
/nfs/latdata/iamjli/projects/GSD/venv/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
04:17:30 - Models: INFO - Run complete, results written to: ../evaluation_clean/results/ICA1_10_0.01/D_5_100_1_Z_20_5_1_noise_0.npy
/nfs/latdata/iamjli/projects/GSD/venv/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:121: ConvergenceWarning: FastICA did not co

### Merge all results

In [9]:
scores = []

for model_params in e.model_paramlist: 
    scores += e.score_results(model_params)
    
scores_df = pd.DataFrame(scores)
scores_df.head()

,F1rr,model,n_samples,n_sources,noise,recovery,relevance,rep,size
0,0.770005,ICA1_10_0.01,20,5,0.0,0.981506,0.633496,1,100
1,0.774671,ICA1_10_0.01,20,5,0.0,0.984502,0.638569,2,100
2,0.118007,ICA1_10_0.01,20,5,0.2,0.222795,0.080259,1,100
3,0.104959,ICA1_10_0.01,20,5,0.2,0.211338,0.069816,2,100
4,0.065736,ICA1_10_0.01,20,5,0.4,0.166867,0.040930,1,100


### Average over replicates

In [10]:
tmp = scores_df.groupby(by=["model", "size", "n_sources", "n_samples", "noise"]).agg([np.mean, np.std]).drop(columns=['rep'])
tmp.head()

F1rr            recovery  \
                                                    mean       std      mean   
model           size n_sources n_samples noise                                 
GSD_0.1_3_ica_5 100  5         20        0.0    0.010304  0.000730  0.009020   
                                         0.2    0.008867  0.000634  0.008750   
                                         0.4    0.008688  0.000563  0.008359   
ICA1_10_0.01    100  5         20        0.0    0.772338  0.003299  0.983004   
                                         0.2    0.111483  0.009227  0.217066   

                                                         relevance            
                                                     std      mean       std  
model           size n_sources n_samples noise                                
GSD_0.1_3_ica_5 100  5         20        0.0    0.000270  0.012206  0.002495  
                                         0.2    0.000452  0.008990  0.000826  
                                         0.4    0.000862  0.009060  0.000209  
ICA1_10_0.01    100  5         20        0.0    0.002119  0.636033  0.003588  
                                         0.2    0.008101  0.075037  0.007384

In [11]:
tmp.query('model=="ICA1_10_0.01"')

F1rr            recovery  \
                                                 mean       std      mean   
model        size n_sources n_samples noise                                 
ICA1_10_0.01 100  5         20        0.0    0.772338  0.003299  0.983004   
                                      0.2    0.111483  0.009227  0.217066   
                                      0.4    0.065479  0.000363  0.172380   

                                                      relevance            
                                                  std      mean       std  
model        size n_sources n_samples noise                                
ICA1_10_0.01 100  5         20        0.0    0.002119  0.636033  0.003588  
                                      0.2    0.008101  0.075037  0.007384  
                                      0.4    0.007797  0.040431  0.000706